In [94]:
import math
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'



In [100]:
test_years = [2015]
input_file = 'logistic_submission_2015.csv'
df = pd.read_csv('../input/NCAATourneyCompactResults.csv')
submission_df = pd.read_csv(input_file)
predictions = {}
for index, row in submission_df.iterrows():
    year, sid, eid = map(int, row['id'].split('_'))
    predictions[(year, sid, eid)] = row['pred']

    
    

In [101]:
def test():
    log_losses = []
    for year in test_years:
        log_loss, ncaa_df = evaluate(year)
        print "Log loss of year {0} is {1}".format(year, log_loss)
        log_losses.append(log_loss)
    average = sum(log_losses) / len(log_losses)
    print "Average log loss across the years {1} - {2} is {0}".format(average, test_years[0], test_years[-1])
    return average, log_losses
test()
    

Log loss of year 2015 is 0.461436112848
Average log loss across the years 2015 - 2015 is 0.461436112848


(0.4614361128483572, [0.4614361128483572])

In [102]:
def predict(row, year):
    mn = min(row['WTeamID'], row['LTeamID'])
    mx = max(row['WTeamID'], row['LTeamID'])
    return predictions[(year, mn, mx)]

def logloss(row):
    return (row.Result * math.log(row.Prediction) + (1.0 - row.Result) * math.log(1.0 - row.Prediction))

def evaluate(year):
    ncaa_df = df[(df['Season'] == year) & (df.DayNum >= 136)]
    assert(len(ncaa_df.index) == 63)
    ncaa_df['Result'] = ncaa_df.apply(lambda x : int(x.WTeamID < x.LTeamID), axis = 1)
    ncaa_df['Prediction'] = ncaa_df.apply(predict, args = (year,), axis = 1)
    ncaa_df['LogLoss'] = ncaa_df.apply(logloss, axis = 1)
    return -(ncaa_df['LogLoss'].sum() / len(ncaa_df.index)), ncaa_df
print(evaluate(2015)[0])

0.4614361128483572
